### ***Installing the Dependencies***

In [ ]:
"""
!pip install fastapi
!pip install uvicorn
!pip install pickle5
!pip install pyngrok
!pip install pydantic
!pip install requests
!pip install pypi-json
!pip install scikit-learn
!pip install nest-asyncio

"""

In [3]:
import json
import pickle
import uvicorn
import nest_asyncio
from pyngrok import ngrok
from fastapi import FastAPI
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware

In [4]:
app = FastAPI()

In [5]:
origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"]
)

In [6]:
class model_input(BaseModel):
    SatisfactionLevel : float
    LastEvaluation : float
    ProjectCount : int
    AverageMonthlyHours : int
    TimeSpent : int
    WorkAccident : int
    PromotionLast_5Years : int
    Salary : int

In [7]:
# loading the saved model
model = pickle.load(open('employee_attrition.sav', 'rb'))

In [8]:
@app.post('/employee_attrition')
def attrition_prediction(input_parameters : model_input):

    input_data = input_parameters.json()
    input_dictionary = json.loads(input_data)

    satisfyLevel = input_dictionary['SatisfactionLevel']
    lastEvaluation = input_dictionary['LastEvaluation']
    projectCount = input_dictionary['ProjectCount']
    avgMonthHours = input_dictionary['AverageMonthlyHours']
    timeSpent = input_dictionary['TimeSpent']
    workAccident = input_dictionary['WorkAccident']
    promotionGiven = input_dictionary['PromotionLast_5Years']
    salary = input_dictionary['Salary']

    input_list = [satisfyLevel, lastEvaluation, projectCount, avgMonthHours, timeSpent, workAccident, promotionGiven, salary]

    prediction = model.predict([input_list])

    if(prediction[0] == 0):
        return "The employee is still working in the organization."
    else:
        return "The employee left the organization."

In [ ]:
ngrok.set_auth_token("2eGt3d5tWley6Yx3hzHTTbzC2d0_5ukXyp7J9Dia55taeNcmt")
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)